<a href="https://colab.research.google.com/github/rostro36/Partisan-Responses/blob/master/QA_Pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir('/content/drive/My Drive/Partisan-Responses-master')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# Install libraries if needed
#! pip install transformers
#! pip install allennlp allennlp-models
#! pip install hnswlib
#! pip install wandb
#! pip install neuralcoref

     |████████████████████████████████| 1.4MB 5.9MB/s 
     |████████████████████████████████| 102kB 11.4MB/s 
     |████████████████████████████████| 163kB 29.8MB/s 
     |████████████████████████████████| 112kB 30.8MB/s 
     |████████████████████████████████| 102kB 11.1MB/s 
     |████████████████████████████████| 71kB 7.9MB/s 
     |████████████████████████████████| 71kB 8.1MB/s 
  Created wheel for subprocess32: filename=subprocess32-3.5.4-cp36-none-any.whl size=6489 sha256=a23745b2f7fe37af7859f052218c52d9e5b7c9dbc3e64b4334aaee5183a85881
  Stored in directory: /root/.cache/pip/wheels/68/39/1a/5e402bdfdf004af1786c8b853fd92f8c4a04f22aad179654d1
  Created wheel for gql: filename=gql-0.2.0-cp36-none-any.whl size=7630 sha256=6ba1a7beb9f36d8272a9d1591f2f1dfff0b1ef88da903c7888d07c72e608eb51
  Stored in directory: /root/.cache/pip/wheels/ce/0e/7b/58a8a5268655b3ad74feef5aa97946f0addafb3cbb6bd2da23
  Created wheel for watchdog: filename=watchdog-0.10.3-cp36-none-any.whl size=73870 sha256=de

Import all libraries

In [2]:
import numpy as np
import pandas as pd
import spacy
import pickle
import torch
import torch.nn.functional as F
from itertools import islice
from transformers import pipeline, GPT2LMHeadModel, GPT2Tokenizer
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from Search import Search
from Answer import Answer
from Speech import Speech
from KnowledgeGraph import KnowledgeGraph
import neuralcoref
import re
import gc
import utils

wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.
INFO:absl:Using /tmp/tfhub_modules to cache modules.
/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: spacy.morphology.Morphology size changed, may indicate binary incompatibility. Expected 104 from C header, got 112 from PyObject
  return f(*args, **kwds)
/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: spacy.vocab.Vocab size changed, may indicate binary incompatibility. Expected 96 from C header, got 104 from PyObject
  return f(*args, **kwds)
/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: spacy.tokens.span.Span size changed, may indicate binary incompatibility. Expected 72 from C header, got 80 from PyObject
  return f(*args, **kwds)


In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

Load data & search

In [ ]:
# Last 14 Congresses
file_name = 'search_dataset_small.pkl'
speeches = pd.read_pickle(file_name)
speeches.head()
speeches = speeches.loc[:1000,:]
#search = Search(speeches=speeches)

In [3]:
search = pickle.load(open("search_results.pkl", "rb"))
search.head()

,question,answer_R,answer_D
0,Should abortion be illegal?,"Mr. President, my amendment simply remedies a ...",Will the Senator yield to answer a question so...
1,What do you believe about tax increases?,"Mr. Speaker, I note that the Speaker of the Ho...",Do you remember the other bill that he signed ...
2,Should same-sex marriage be legal?,Then I will submit all this for the RECORD and...,"Mr. Speaker, I yield myself such time as I may..."
3,Is climate change real?,"Mr. President, this amendment addresses the la...","Mr. Speaker, I thank the distinguished gentlem..."
4,Should immigrants be allowed to obtain citizen...,"Mr. Chairman, as a strong advocate of immigrat...","Mr. President, I am pleased to introduce legis..."


In [4]:
# Manual Questions (not from corpus)
questions = ["Should abortion be illegal?",
             "What do you believe about tax increases?",
             "Should same-sex marriage be legal?",
             "Is climate change real?",
             "Should immigrants be allowed to obtain citizenship?",
             "Who should be given voting rights?",
             "Should we have higher taxes for higher incomes?",
             "Should we allow death penalty?",
             "Should we have universal healthcare?",
             "Do government regulations hinder free market capitalism?",
             "What do you think about the current president?",
             "Should we reduce national debt?",
             "Should we increase spending on healthcare?",
             "Should we increase spending on education?",
             "Should every American have equal opportunities regardless of sex, age and race?",
             "Should be introduce more gun control measures?",
             "What do you think about the current president?",
             "Should Americans be free?",
             "What party do you support?",
             "What is the biggest threat to America?"
             ]

Graph construction

In [5]:
#from KnowledgeGraph import KnowledgeGraph
knowledgeGraphs=dict()
for question in questions:
  knowledgeGraphs[question]=KnowledgeGraph(question)

In [6]:
identifier='coref'
checkpoint=12
#verb_list_file = "verb_list"+identifier+str(checkpoint)+".pickle"
verb_dict = pickle.load(open("verb_dict.pickle", "rb"))
verb_list = None #pickle.load(verb_list_file)
graphWriterData=[]

In [7]:
def parse_entry(question,answer,verb_dict,verb_list):
    result=dict()
    result['question']=' '.join([token.text for token in utils.sp(question)])
    phrase_corpus, triplet_id, parsed_text,parsed=Answer(answer).create_test(verb_dict,verb_list)
    result['corpus']=' ; '.join(phrase_corpus)
    result['tags']=' '.join(['<phrase>']*len(phrase_corpus))
    result['triplet_id']=' ; '.join([re.sub('\,','',str(x))[1:-1] for x in triplet_id])
    result['parsed_text']=parsed_text
    result['parsed']=' '.join([str(x) for x in parsed])
    return result

Generate answers to question with GPT-2

In [8]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained('gpt2')

In [9]:
for question in questions:
    print("Question: {}".format(question))
    print("GPT-2 generation: ")
    generated = tokenizer.encode(question)
    context = torch.tensor([generated])
    past = None

    for i in range(100):
        output, past = model(context, past=past)
        token = torch.argmax(output[..., -1, :])

        generated += [token.tolist()]
        context = token.unsqueeze(0)

    sequence = tokenizer.decode(generated)

    print(sequence)

Question: Should abortion be illegal?
GPT-2 generation: 
Should abortion be illegal?

The Supreme Court has ruled that abortion is illegal under the Fourteenth Amendment. The Supreme Court has ruled that abortion is illegal under the Fourteenth Amendment.

The Supreme Court has ruled that abortion is illegal under the Fourteenth Amendment.

The Supreme Court has ruled that abortion is illegal under the Fourteenth Amendment.

The Supreme Court has ruled that abortion is illegal under the Fourteenth Amendment.

The Supreme Court has ruled that abortion is illegal under the Fourteenth Amendment.
Question: What do you believe about tax increases?
GPT-2 generation: 
What do you believe about tax increases?

I think that the tax increases are going to be very good for the economy. I think that the tax increases are going to be very good for the middle class. I think that the middle class is going to be very happy. I think that the middle class is going to be very happy.

I think that the mid

Search

In [19]:
from KnowledgeGraph import KnowledgeGraph
for i in range(len(search)):
    question = search.iloc[i].question
    print("Question: {}".format(question))
    answer_R = search.iloc[i].answer_R
    print("Republican Result: {}".format(answer_R))
    answer_D = search.iloc[i].answer_D
    print("Democrat Result: {}".format(answer_D))
    
    res_R = pd.Series({'speech': answer_R, 'party': 'R'})
    res_D = pd.Series({'speech': answer_D, 'party': 'D'})
    for res in [res_R, res_D]:
        print(res)
        triplets=Speech(res).create_triplet()
        knowledgeGraphs[question].add_edges(triplets)

        parsed = parse_entry(question,full_result,verb_dict,verb_list)
        graphWriterData.append(parsed)

Question: Should abortion be illegal?
Republican Result: Mr. President, my amendment simply remedies a major defect in this bill by ensuring that it does not cover illegal abortions. Why not limit protections of this bill to lawful abortions? I cannot imagine any rationale that could be used to rabut the import of that question. This whole debate shows how extreme this bill is on the proabortion side, I think it would have a lot more support if it was not so extreme, if it did not rush to support illegal abortions and illegal abortionists, to avoid the mere risk of abusive discovery, which is about the only argument they can make. That is a risk every litigant faces, I have been in all kinds of litigation in my lifetime as an attorney. Every case involves the potential abuse of discovery. But to use that as an excuse to not knock out illegal abortions in this bill shows how extreme this bill is. S, 636 very simply protects illegal abortion. It is that simple. Why is it so difficult to 

RuntimeError: ignored

In [ ]:
!rm GraphWriter-master/data/preprocessed.test.tsv

In [ ]:
m=pd.DataFrame(data=graphWriterData)
m.to_csv('GraphWriter-master/data/preprocessed.test.tsv', sep='\t', index=False, header=False)

In [ ]:
for knowledgeGraph in knowledgeGraphs:
  knowledgeGraph.draw()

Text Generation via GraphWriter

In [ ]:
!python GraphWriter-master/generator.py -save=GraphWriter-master/partisan-responses/2.vloss-3.949412.lr-0.05